In [45]:
from random import random, randrange, shuffle
import pandas
import csv
from numpy import mean
import matplotlib.pyplot as plt
import math



def mutate(child):
    mutate1 = randrange(0,24) 
    mutate2 = randrange(0,24) 
    while mutate2 == mutate1:
        mutate2 = randrange(0,24)
    if mutate1 != 0:
        if child[mutate1] == "0":
            child = child[0:mutate1] + '1' + child[mutate1+1:]
        else:
            child = child[0:mutate1] + '0' + child[mutate1+1:]
    else:
        if child[mutate1] == "0":
            child = '1' + child[1:]
        else:
            child = '0' + child[1:]
    
    if mutate2 != 0:
        if child[mutate2] == "0":
            child = child[0:mutate2] + '1' + child[mutate2+1:]
        else:
            child = child[0:mutate2] + '0' + child[mutate2+1:]
    else:
        if child[mutate2] == "0":
            child = '1' + child[1:]
        else:
            child = '0' + child[1:]
    return child        

def crossover(chromosome):
    for i in range(30,len(chromosome),2):
        parent1 = chromosome[i].chromosome
        parent2 = chromosome[i+1].chromosome
#         print("parent 1 " + parent1)
#         print("parent 2 " + parent2)
        child1 = parent1[0:4] + parent2[4:8] + parent1[8:12] + parent2[12:16] + parent1[16:20] + parent2[20:24]
        child2 = parent2[0:4] + parent1[4:8] + parent2[8:12] + parent1[12:16] + parent2[16:20] + parent1[20:24]
#         print("child 1 " + child1)
#         print("child 2 " + child2)
        child1 = mutate(child1)
        child2 = mutate(child2)
#         print("child 1 after mutate " + child1)
#         print("child 2 after mutate " + child2)
        chromosome[i].chromosome = child1
        chromosome[i].chromosome = child2
    return chromosome
    
def create_chromosome(data, bestLn, avgLn) :
    class Chromosome():
        chromosome =''
        fitness = 0
    temp = Chromosome()
    best = 0
    chromosome = []
    sumFitness = 0
    for i in range(32):
        gene = str(randrange(0,2))
        for j in range(23):
            gene = gene + str(randrange(0,2))
        temp.chromosome = gene
        mealData1 = accessData(data ,temp.chromosome[0:8])
        mealData2 = accessData(data ,temp.chromosome[8:16])
        mealData3 = accessData(data ,temp.chromosome[16:])
        while not checkCons(mealData1, mealData2, mealData3):
            temp.chromosome = mutate(temp.chromosome)
            mealData1 = accessData(data ,temp.chromosome[0:8])
            mealData2 = accessData(data ,temp.chromosome[8:16])
            mealData3 = accessData(data ,temp.chromosome[16:])
        temp.fitness = fitnessFn(mealData1, mealData2, mealData3)
        sumFitness += temp.fitness
        if best < temp.fitness:
            best = temp.fitness   
        chromosome.append(temp)
        temp = Chromosome()
        gene = ''
    bestLn.append(best)
    avgLn.append(sumFitness/len(chromosome))
    return chromosome
        
def fitnessFn(meal1, meal2, meal3):
    totalProtein = meal1['protein'] + meal2['protein'] + meal3['protein']
    totalCalories = meal1['calories'] + meal2['calories'] + meal3['calories']
    return 0.5*totalProtein + 0.5*totalCalories
#     return 0.5*totalProtein 
    
def breeding(data, chromosome, badChromosome, bestLn, avgLn):
    sumFitness = 0
    best = 0
    chromosome = sorted(chromosome, key=lambda x: x.fitness, reverse=True)
    chromosome = crossover(chromosome)
    for cms in chromosome:
        while cms.chromosome in badChromosome:
            cms.chromosome = mutate(cms.chromosome)
            print('detect bad chr!')
        while cms.chromosome[0:8] == cms.chromosome[8:16] or cms.chromosome[0:8] == cms.chromosome[16:] or cms.chromosome[8:16] == cms.chromosome[16:]:
            cms.chromosome = mutate(cms.chromosome)
            print('same menu!')
        mealData1 = accessData(data ,cms.chromosome[0:8])
        mealData2 = accessData(data ,cms.chromosome[8:16])
        mealData3 = accessData(data ,cms.chromosome[16:])
        while not checkCons(mealData1, mealData2, mealData3):
            if cms.chromosome not in badChromosome:
                badChromosome.append(cms.chromosome) 
            cms.chromosome = mutate(cms.chromosome)
            mealData1 = accessData(data ,cms.chromosome[0:8])
            mealData2 = accessData(data ,cms.chromosome[8:16])
            mealData3 = accessData(data ,cms.chromosome[16:])
        cms.fitness = fitnessFn(mealData1, mealData2, mealData3)
        sumFitness += cms.fitness
        if best < cms.fitness:
            best = cms.fitness
    bestLn.append(best)
    avgLn.append(sumFitness/len(chromosome))
    print('best: ' + str(best))
    print('avg: ' + str(sumFitness/len(chromosome)))
    print('bad chromosome: ' + str(len(badChromosome)))
    print('-----------------------')
        

def checkCons(foodData1,foodData2,foodData3) :
    if(foodData1['sugar'] + foodData2['sugar'] + foodData3['sugar'] >= 24 and foodData1['sugar'] + foodData2['sugar'] + foodData3['sugar'] <= 50) :
        if(foodData1['protein'] + foodData2['protein'] + foodData3['protein'] >= 75 and foodData1['protein'] + foodData2['protein'] + foodData3['protein'] <= 175) :
            if(foodData1['calories'] + foodData2['calories'] + foodData3['calories'] >= 1500 and foodData1['calories'] + foodData2['calories'] + foodData3['calories'] <= 2500) :
                if(foodData1['carb'] + foodData2['carb'] + foodData3['carb']  >= 150 and foodData1['carb'] + foodData2['carb'] + foodData3['carb'] <= 200) :
                    if(foodData1['fat'] + foodData2['fat'] + foodData3['fat'] >= 50 and foodData1['fat'] + foodData2['fat'] + foodData3['fat'] <= 70) :
                        return True
    return False

def binToDec(binary):
    dec = 0
    for i in range(0, len(binary)):
        temp = int(binary[i])*(2**(len(binary)-i-1))
        dec += temp
    return dec


# input name of dataframe and 8-bit chromosome
def accessData(df, chromosome) :
    menuNumber = binToDec(chromosome)
    sugar = df.Sugar[menuNumber]
    protein = df.Protein[menuNumber]
    calories = df.Calories[menuNumber]
    carb = df.Carbohydrates[menuNumber]
    fat = df.Total_Fat[menuNumber]
    name = df.Item_Name[menuNumber]
    return {"name": name, "sugar": sugar, "protein": protein, "calories": calories, "carb": carb, "fat": fat}



chromosome = []
badChromosome = []
bestLn = []
avgLn = []
genNumber = list(range(1))
di = r"sampling_Filtered_Nutriton.csv"
article_read = pandas.read_csv(di, encoding='latin1')
# article_read = article_read.sort_values(['Protein','Calories'], ascending=[False,False])
chromosome = create_chromosome(article_read, bestLn, avgLn)
gen = 0
for i in genNumber:
    breeding(article_read, chromosome, badChromosome, bestLn, avgLn)
    gen = gen + 1
    if 0.05 * bestLn[-1] < abs(bestLn[-1] - avgLn[-1]):
        genNumber.append(i+1)
        print(genNumber[-1])
#     print(bestLn)
#     print(avgLn)
gen = gen + 1
genLn = list(range(0,gen))
plt.plot(genLn, bestLn, label = "best line") 
plt.plot(genLn, avgLn, label = "average line") 
plt.xlabel('generation') 
plt.ylabel('fitness') 
plt.title('Genetic Algorithm') 
plt.legend() 
plt.show() 
chromosome = sorted(chromosome, key=lambda x: x.fitness, reverse=True)
i = 1
for show in chromosome:
    firstMenu = accessData(article_read, show.chromosome[0:8])['name']
    secondMenu = accessData(article_read, show.chromosome[8:16])['name']
    thirdMenu = accessData(article_read, show.chromosome[16:])['name']
    print('----------- ' + str(i) + ' ------------')
    print(firstMenu)
    print(secondMenu)
    print(thirdMenu)
    print("fitness: " + str(show.fitness))
    i = i + 1






same menu!
best: 1097.5
avg: 886.96875
bad chromosome: 428
-----------------------
1
best: 1097.5
avg: 890.0625
bad chromosome: 429
-----------------------
2
best: 1097.5
avg: 894.125
bad chromosome: 593
-----------------------
3
best: 1097.5
avg: 894.65625
bad chromosome: 723
-----------------------
4
best: 1097.5
avg: 897.3125
bad chromosome: 725
-----------------------
5
best: 1097.5
avg: 896.671875
bad chromosome: 735
-----------------------
6
best: 1097.5
avg: 896.640625
bad chromosome: 1006
-----------------------
7
best: 1097.5
avg: 900.140625
bad chromosome: 1128
-----------------------
8
same menu!
same menu!
best: 1097.5
avg: 904.6875
bad chromosome: 1185
-----------------------
9
best: 1097.5
avg: 903.640625
bad chromosome: 1235
-----------------------
10
best: 1097.5
avg: 909.609375
bad chromosome: 1261
-----------------------
11
best: 1097.5
avg: 912.25
bad chromosome: 1615
-----------------------
12
best: 1097.5
avg: 911.15625
bad chromosome: 1866
-----------------------


KeyboardInterrupt: 